In [18]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Cluster Analysis/Output')
cur_date = "040622"

library(readxl)
library(tidyverse)
library(reshape2)
library(factoextra)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_040323.xlsx", sheet = 2))
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))

In [2]:
head(mRNA_df)
head(proteomics_25_df)

,Subject_No,Subject_ID,Gene_Name,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


,Original_Sample_ID,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,202220317-22-D1-CF25,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1.49e+09,30.47267
2,202220317-32-D2-CF25,2,M_2,K1C19_HUMAN,KRT19,"Keratin, type I cytoskeletal 19 OS=Homo sapiens OX=9606 GN=KRT19 PE=1 SV=4",C,F,25,6.02e+09,32.48712
3,202220317-28-D3-CF25,3,M_3,TBB4B_HUMAN,TUBB4B,Tubulin beta-4B chain OS=Homo sapiens OX=9606 GN=TUBB4B PE=1 SV=1,C,F,25,3.07e+09,31.51559
4,202220317-08-D4-CF25,4,F_4,K2C5_HUMAN,KRT5,"Keratin, type II cytoskeletal 5 OS=Homo sapiens OX=9606 GN=KRT5 PE=1 SV=3",C,F,25,4.77e+09,32.15134
5,202220317-18-D5-CF25,5,F_5,ACTG_HUMAN,ACTG1,"Actin, cytoplasmic 2 OS=Homo sapiens OX=9606 GN=ACTG1 PE=1 SV=1",C,F,25,5.42e+09,32.33565
6,202220317-12-D6-CF25,6,M_6,PIGR_HUMAN,PIGR,Polymeric immunoglobulin receptor OS=Homo sapiens OX=9606 GN=PIGR PE=1 SV=4,C,F,25,8.04e+09,32.90455


This analysis will use PCA & k means to explore to determine if each biomarker clusters into groups.

In [11]:
# pivoting longer but first creating sample ids
wider_mRNA_df = mRNA_df %>%
    # creating a column that denotes the biomarker
    mutate(Biomarker = "mRNA") %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(mRNA_df)[c(4:7)], "Biomarker")) %>%
    select(-c("Subject_No", "ddCT")) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "ddCT_pslog2")

head(wider_mRNA_df)

Warning message:
“Values from `ddCT_pslog2` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} %>%
    dplyr::group_by(Gene_Name, Sample_ID, Subject_ID) %>%
    dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
    dplyr::filter(n > 1L)”


Gene_Name,Sample_ID,F_1,M_2,M_3,F_4,M_6,F_5
<chr>,<chr>,<list>,<list>,<list>,<list>,<list>,<list>
ALDH1A3,C_F_1_24_mRNA,4.480105,4.870661,4.879307,4.878692,4.850246,NULL
ALDH1A3,C_F_1_72_mRNA,4.8474,4.853069,4.850874,4.881133,4.844027,4.877173
ALDH1A3,C_S_1_24_mRNA,NULL,4.842062,4.869919,4.863741,4.91511,4.902314
ALDH1A3,C_S_1_72_mRNA,4.875436,4.885058,4.868836,4.858839,4.885779,4.879357
ALDH1A3,C_F_25_4_mRNA,"4.850829, 4.843997","4.858401, 4.862782","4.898830, 4.887884","4.861283, 4.873939","4.861486, 4.857343",NULL
ALDH1A3,C_F_25_24_mRNA,4.891977,4.881552,4.863584,4.861217,4.875984,4.877587
ALDH1A3,C_F_25_72_mRNA,4.904412,4.89836,4.849387,4.874381,4.865843,4.868691
ALDH1A3,C_S_25_4_mRNA,"4.910852, 4.895969","4.864358, 4.861943","4.891048, 4.885630","4.867805, 4.871020","4.892004, 4.882764","4.860395, 4.862442"
ALDH1A3,C_S_25_24_mRNA,4.865115,4.842453,4.851229,4.913267,4.851951,4.898058


In [23]:
proteomics_25_df %>%
    # creating a column that denotes the biomarker
    mutate(Biomarker = "Protein") %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(proteomics_25_df)[c(4,7:9)], "Biomarker")) %>%
    select(-colnames(proteomics_25_df)[c(1,2,5,6,10)]) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "Intensity_pslog2")

Warning message:
“Values from `Intensity_pslog2` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} %>%
    dplyr::group_by(Sample_ID, Subject_ID) %>%
    dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
    dplyr::filter(n > 1L)”


Sample_ID,F_1,M_2,M_3,F_4,F_5,M_6
<chr>,<list>,<list>,<list>,<list>,<list>,<list>
CO3_HUMAN_C_F_25_Protein,"30.47267, 31.49667",NULL,"31.82714, 32.07048",NULL,"31.86083, 31.09270",NULL
K1C19_HUMAN_C_F_25_Protein,NULL,"32.48712, 34.01837",NULL,"34.19748, 31.97117",NULL,"31.95394, 31.61982"
TBB4B_HUMAN_C_F_25_Protein,"30.81574, 30.55728",NULL,"31.51559, 33.77510",NULL,"33.79459, 30.37244",NULL
K2C5_HUMAN_C_F_25_Protein,NULL,"31.70058, 31.36202",NULL,"32.15134, 34.06728",NULL,"31.70471, 31.36724"
ACTG_HUMAN_C_F_25_Protein,"33.81383, 32.30606",NULL,"31.68812, 31.84969",NULL,"32.33565, 34.30335",NULL
PIGR_HUMAN_C_F_25_Protein,NULL,"34.15285, 33.15134",NULL,"33.34361, 33.11768",NULL,"32.90455, 33.87921"
H4_HUMAN_C_F_25_Protein,"32.29789, 34.36333",NULL,"34.86083, 32.86268",NULL,"33.07048, 31.78877",NULL
ALBU_HUMAN_C_F_25_Protein,NULL,"29.94000, 33.30335",NULL,"33.98482, 32.46538",NULL,"35.26891, 30.45317"
AL1A1_HUMAN_C_F_25_Protein,"31.33031, 30.98821",NULL,"30.93298, 34.01005",NULL,"34.63071, 31.39305",NULL
